In [ ]:
# Installer les bibliothèques nécessaires
!pip install tensorflow spacy
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

In [ ]:

# Importer les bibliothèques
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
# Charger les modèles de langage français et anglais
modele_francais = spacy.load('fr_core_news_sm')
modele_anglais = spacy.load('en_core_web_sm')

In [ ]:
# Charger le dataset
dataset_url = "https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench"
data = pd.read_csv(dataset_url, delimiter="\t", header=None).head(500)

In [ ]:
doc.shape

In [ ]:
# Fusionner les phrases pour former le corpus complet en français et en anglais
corpus_francais = " ".join(data.iloc[:, 1])
corpus_anglais = " ".join(data.iloc[:, 0])

In [ ]:
# Tokeniser les phrases avec spacy
data["tokens_fr"] = data.iloc[:, 1].apply(modele_francais)
data["tokens_en"] = data.iloc[:, 0].apply(modele_anglais)

In [ ]:
# Obtenir le vocabulaire unique pour chaque langue
vocabulaire_fr = set([token.text for doc in data["tokens_fr"] for token in doc])
vocabulaire_en = set([token.text for doc in data["tokens_en"] for token in doc])

In [ ]:
# Créer des dictionnaires de tokens vers index pour chaque vocabulaire
index_fr = {token: i + 1 for i, token in enumerate(vocabulaire_fr)}
index_en = {token: i + 1 for i, token in enumerate(vocabulaire_en)}

In [ ]:
# Fonctions pour convertir des séquences de tokens en indices
def convertir_tokens_fr(tokens):
    return [index_fr.get(token.text, 0) for token in tokens]

def convertir_tokens_en(tokens):
    return [index_en.get(token.text, 0) for token in tokens]

In [ ]:
# Appliquer la conversion des tokens en indices
data["indices_fr"] = data["tokens_fr"].apply(convertir_tokens_fr)
data["indices_en"] = data["tokens_en"].apply(convertir_tokens_en)

In [ ]:
# Utiliser Keras pour uniformiser la longueur des séquences
sequence_fr = tf.keras.preprocessing.sequence.pad_sequences(data["indices_fr"], padding="post")
sequence_en = tf.keras.preprocessing.sequence.pad_sequences(data["indices_en"], padding="post")

In [ ]:
# Taille du vocabulaire
taille_vocabulaire_fr = len(index_fr)
taille_vocabulaire_en = len(index_en)

# Définir le modèle
modele_seq2seq = tf.keras.Sequential([
    # Couche d'Embedding pour les mots
    tf.keras.layers.Embedding(input_dim=taille_vocabulaire_fr + 1, output_dim=128, mask_zero=True),

    # Couche LSTM bidirectionnelle
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),

    # Deuxième couche LSTM bidirectionnelle
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=False)),

    # Répéter le vecteur pour correspondre aux séquences de sortie
    tf.keras.layers.RepeatVector(sequence_en.shape[1]),

    # Couche LSTM supplémentaire pour le décodeur
    tf.keras.layers.LSTM(64, return_sequences=True),

    # Couche de sortie avec Softmax pour la classification des tokens
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(taille_vocabulaire_en + 1, activation="softmax"))
])

# Compiler le modèle
modele_seq2seq.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entraînement du modèle
historique = modele_seq2seq.fit(sequence_fr, sequence_en, epochs=10, batch_size=32, validation_split=0.2)

# Évaluation du modèle sur les données d'entraînement
evaluation = modele_seq2seq.evaluate(sequence_fr, sequence_en)
print(f"Précision d'évaluation : {evaluation[1] * 100:.2f}%")


In [ ]:
modele_seq2seq.summary()

In [ ]:
#model.save("translation_model.h5")
modele_seq2seq.save('translation_model.keras')

In [ ]:
# Tester une prédiction avec des données
def tester_traduction(entree_francais):
    # Tokeniser et encoder la phrase d'entrée
    tokens = modele_francais(entree_francais)
    sequence = convertir_tokens_fr(tokens)
    sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=sequence_fr.shape[1], padding="post")

    # Générer la prédiction
    prediction = modele_seq2seq.predict(sequence)
    prediction_indices = np.argmax(prediction[0], axis=-1)

    # Décoder la séquence prédite en mots anglais
    mots_predits = [list(index_en.keys())[list(index_en.values()).index(i)] for i in prediction_indices if i in index_en.values()]
    traduction = " ".join(mots_predits)

    print(f"Traduction prédite : {traduction}")

# Exemple de test de traduction
tester_traduction("Bonjour, comment allez-vous ?")
